In [159]:
import requests
import pandas as pd
import requests 
import os

In [161]:
url = 'https://op.mos.ru/EHDWSREST/catalog/export/get?id='
ids = [785561, 785489, 785513, 785597, 785477, 784213]

In [162]:
temp_file = 'temp.zip'

In [163]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

# calendar.txt

In [219]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(ids[0]), temp_file)

In [222]:
a = pd.read_csv(temp_file, sep=';', encoding='cp1251')
a = a[a.global_id != 'global_id']
a[['service_id', 'monday', 'tuesday', 'wednesday', 'thursday',
   'friday', 'saturday', 'sunday', 'start_date',
   'end_date']].to_csv('data/gtfs/calendar.txt', sep=',', index=False)
a.head(2)

,global_id,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,Unnamed: 11
0,833044189,3,1,1,1,1,1,0,0,20190501,20190930,NaN
1,833044708,4,0,0,0,0,0,1,1,20190501,20190930,NaN


# routes.txt

In [224]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(ids[1]), temp_file)

In [225]:
a = pd.read_csv(temp_file, sep=';', encoding='cp1251')
a = a[a.global_id != 'global_id']
a = a[['route_id', 'agency_code', 'route_short_name', 'route_long_name', 'route_type', 'route_desc']]
a = a.rename(columns = {'agency_code':'agency_id'})
a.route_type.unique()

array(['3', '5', '0'], dtype=object)

In [226]:
a.route_type = a.route_type.replace('5', '11')

In [227]:
a.to_csv('data/gtfs/routes.txt', sep=',', index=False)
a.head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_desc
0,14150158,1,597,"Метро ""Сокол"" - Набережная Новикова-Прибоя",3,NaN
1,14150132,2,380,Ст. МЦД Дегунино - Пр. Русанова,3,NaN


# stops.txt

In [210]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(ids[2]), temp_file)
a = pd.read_csv(temp_file, sep=';', encoding='cp1251')
a = a[a.global_id != 'global_id']
a['location_type'] = 0
a.head(2)

,global_id,stop_id,stop_name,StationName,Street,Direction,Pavilion,OperatingOrgName,geoData,Unnamed: 9,location_type
0,833043944,3630,5-я Парковая ул.,NaN,NaN,NaN,NaN,NaN,"{type=Point, coordinates=[37.78753642, 55.7929...",NaN,0
1,833043948,3631,7-я Парковая ул.,NaN,NaN,NaN,NaN,NaN,"{type=Point, coordinates=[37.79377237, 55.7930...",NaN,0


In [211]:
coords = [x.split('[')[1][:-2].split(', ') for x in a.geoData]
a['stop_lon'] = [float(x[0]) for x in coords]
a['stop_lat'] = [float(x[1]) for x in coords]
a = a[['stop_id', 'stop_name', 'stop_lon', 'stop_lat', 'location_type']]

In [216]:
a.to_csv('data/gtfs/stops.txt', sep=',', index=False)

# stop_times.txt

In [255]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(ids[3]), temp_file)
a = pd.read_csv(temp_file, sep=';', encoding='cp1251')
a = a[a.global_id != 'global_id']
a.head()

/usr/local/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,global_id,stop_times_id,trip_id,arrival_time,departure_time,stop_id,shape_dist_traveled,pickup_type,stop_headsign,stop_sequence,trip_type,stop_interval,Unnamed: 12
0,870892967,NaN,14150167_1_4_2,07:10:00,07:10:00,1002227,NaN,NaN,NaN,1,NaN,NaN,NaN
1,870892968,NaN,14150167_1_3_32,22:33:00,22:33:00,1001069,NaN,NaN,NaN,13,NaN,NaN,NaN
2,870892970,NaN,14150167_1_4_2,07:12:00,07:12:00,11260,NaN,NaN,NaN,2,NaN,NaN,NaN
3,870892971,NaN,14150167_1_3_33,22:40:00,22:40:00,1001069,NaN,NaN,NaN,1,NaN,NaN,NaN
4,870892972,NaN,14150167_1_3_33,22:41:00,22:41:00,1002157,NaN,NaN,NaN,2,NaN,NaN,NaN


In [256]:
a[['trip_id', 'arrival_time',
  'departure_time', 'stop_id', 'stop_sequence',
   'stop_headsign', 'pickup_type']].to_csv('data/gtfs/stop_times.txt', sep=',', index=False)

# trips.txt


In [257]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(ids[4]), temp_file)
a = pd.read_csv(temp_file, sep=';', encoding='cp1251')
a = a[a.global_id != 'global_id']
a.head()

,global_id,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,volume_id,trip_type,Unnamed: 9
0,870379299,14150198,4,14150198_4_3_9,NaN,NaN,NaN,NaN,NaN,NaN
1,870379300,14150198,4,14150198_4_4_10,NaN,NaN,NaN,NaN,NaN,NaN
2,870379301,14150198,4,14150198_4_4_11,NaN,NaN,NaN,NaN,NaN,NaN
3,870379302,14150198,4,14150198_4_3_10,NaN,NaN,NaN,NaN,NaN,NaN
4,870379303,14150198,4,14150198_4_4_12,NaN,NaN,NaN,NaN,NaN,NaN


In [258]:
a[['route_id', 'trip_id', 'service_id',
   'trip_headsign', 'block_id']].to_csv('data/gtfs/trips.txt', sep=',', index=False)

# agency.txt

In [263]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(ids[5]), temp_file)
a = pd.read_csv(temp_file, sep=';', encoding='cp1251')
a = a[a.global_id != 'global_id']
a = a.rename(columns = {'agency_code':'agency_id'})
a[['agency_id', 'agency_name',
   'agency_url', 'agency_timezone']].to_csv('data/gtfs/agency.txt', sep=',', index=False)

/usr/local/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [265]:
os.remove(temp_file)